# Exercise: FAS multigrid for (\nabla^2 u + \sigma u^2 = f)

## 1) Choose an exact solution (for testing)

Pick a known $u_{\text{exact}}$ and build $f$ so the PDE holds:

Let $u_{\text{exact}}(x,y) = \sin(\pi x)\sin(\pi y)$. Then
$$
\Delta u_{\text{exact}} = -2\pi^2 \sin(\pi x)\sin(\pi y) = -2\pi^2 u_{\text{exact}}.
$$
Thus choose
$$
f(x,y) = \Delta u_{\text{exact}} + \sigma,u_{\text{exact}}^2 = -2\pi^2 u_{\text{exact}} + \sigma u_{\text{exact}}^2.
$$
Boundary conditions: $u=0$ on the boundary (Dirichlet), which matches the sine product.

---

## 2) Discretization (same grid as before)

Interior grid size $N\times N$, spacing $h=1/(N+1)$. Discrete Laplacian at $(i,j)$:
$$
(\Delta_h u)_{i,j} = \frac{u_{i+1,j}+u_{i-1,j}+u_{i,j+1}+u_{i,j-1}-4u_{i,j}}{h^2}.
$$

Nonlinear residual at a fine grid point:
$$
R_{i,j}(u) = (\Delta_h u)_{i,j} + \sigma,u_{i,j}^2 - f_{i,j}.
$$

---

## 3) Nonlinear smoother: pointwise Newton (nonlinear Gauss-Seidel)

At each interior point treat neighbors fixed and solve for $u_{i,j}$ using one Newton update:

* residual $r = (\Delta_h u)_{i,j} + \sigma u_{i,j}^2 - f_{i,j}$
* derivative $w.r.t. (u_{i,j})$: $\partial r/\partial u_{i,j} = -4/h^2 + 2\sigma u_{i,j}$
* Newton update: $u_{i,j} \leftarrow u_{i,j} - r / (\partial r/\partial u_{i,j})$  
  Optionally damp the update with factor $0<\omega\le1$: $u\leftarrow u - \omega\cdot r / (\partial r/\partial u_{i,j})$.

Sweep over points in Gauss-Seidel order (e.g., lexicographic or red-black).

---

## 4) FAS V-cycle (two-grid flavor)

Given fine-grid current approximation $u^h$, RHS $f^h$:

1. Pre-smooth: apply $\nu_1$ nonlinear smoothing sweeps on $u^h$.
2. Compute fine-level residual $r^h = f^h - L^h(u^h)$
   (here $L^h(u) = \Delta_h u + \sigma u^2$).
3. Restrict current approximation and residual to coarse grid:

   * $u^{2h}_0 = R(u^h)$ (restrict the **approximation**)
   * define coarse RHS (FAS tau-correction):
     $$
     f^{2h} = R(f^h) + L^{2h}(u^{2h}_0) - R(L^h(u^h))
     $$
     This ensures coarse-grid problem has the same correction effect as the fine grid.
4. Solve the coarse-grid nonlinear problem for $u^{2h}$ (recursively apply FAS or do many smoothing sweeps / Newton).
5. Prolongate correction and update:
   $$
   u^h \leftarrow u^h + P\bigl(u^{2h} - u^{2h}_0\bigr)
   $$
6. Post-smooth: apply $\nu_2$ nonlinear smoothing sweeps on $u^h$.

Important: FAS transfers **approximation** $u$ (not residual directly), and includes the tau-correction term $L^{2h}(u^{2h}_0) - R(L^h(u^h))$.

---

## 5) Diagnostics

Track:

* Nonlinear residual norm: $|R(u)|_2$.
* Error norm $|u - u_{\text{exact}}|_2$ (since we know exact solution).
* Visualize error/residual heatmaps before/after cycles and smoothing.

# Step1: setup

In [4]:
using LinearAlgebra, Plots

N = 64
h = 1.0/(N+1)
x = range(h, step=h, length=N)
y = range(h, step=h, length=N)

u_exact(x,y) = sin(pi*x)*sin(pi*y)
σ = 5.0   # arbitrarily set, can be changed to see various "strength" of the non-linear term
f = [ -2*pi^2 * u_exact(xi,yj) + σ * u_exact(xi,yj)^2 for xi in x, yj in y ]
u_true = [ u_exact(xi,yj) for xi in x, yj in y ]

64×64 Matrix{Float64}:
 0.00233418  0.00466291  0.00698075  …  0.00698075  0.00466291  0.00233418
 0.00466291  0.00931494  0.0139452      0.0139452   0.00931494  0.00466291
 0.00698075  0.0139452   0.0208771      0.0208771   0.0139452   0.00698075
 0.00928229  0.0185429   0.0277602      0.0277602   0.0185429   0.00928229
 0.0115621   0.0230973   0.0345785      0.0345785   0.0230973   0.0115621
 0.013815    0.0275977   0.041316    …  0.041316    0.0275977   0.013815
 0.0160356   0.0320337   0.0479571      0.0479571   0.0320337   0.0160356
 0.0182187   0.0363949   0.0544861      0.0544861   0.0363949   0.0182187
 0.0203593   0.0406711   0.0608878      0.0608878   0.0406711   0.0203593
 0.0224523   0.0448523   0.0671474      0.0671474   0.0448523   0.0224523
 0.0244929   0.0489287   0.0732501   …  0.0732501   0.0489287   0.0244929
 0.0264763   0.0528908   0.0791818      0.0791818   0.0528908   0.0264763
 0.0283979   0.0567295   0.0849285      0.0849285   0.0567295   0.0283979
 ⋮          